In [3]:
import pandas as pd
import plotly.graph_objs as go

# Загрузка и предобработка данных
data = pd.read_csv(
    r'data\directMeasurement.csv',
    sep=','
)

df = data.copy(deep=True)

df = df.drop('id', axis=1)
for col in df.columns:
    df[col] = df[col].apply(lambda x: float(f'0.{x}'))
new_data = df.transpose()

# Нахождение статистических показателей
new_data['mean_up'] = df.iloc[:3, :].mean() # Для хода поплавка вверх
new_data['mean_down'] = df.iloc[3:, :].mean() # Для хода поплавка вниз
new_data['std'] = df.std()
new_data['error'] = abs(new_data['mean_down'] - new_data['mean_up']) # Погрешность для разных проходов  
    
new_data

,0,1,2,3,4,5,mean_up,mean_down,std,error
10,0.104,0.104,0.104,0.104,0.104,0.104,0.104000,0.104000,0.000000,0.000000
20,0.193,0.202,0.197,0.195,0.198,0.198,0.197333,0.197000,0.003061,0.000333
30,0.288,0.295,0.304,0.297,0.300,0.298,0.295667,0.298333,0.005367,0.002667
40,0.391,0.397,0.397,0.397,0.395,0.400,0.395000,0.397333,0.002994,0.002333
50,0.493,0.497,0.495,0.493,0.500,0.502,0.495000,0.498333,0.003724,0.003333
60,0.596,0.595,0.598,0.595,0.593,0.598,0.596333,0.595333,0.001941,0.001000
70,0.698,0.695,0.695,0.698,0.698,0.698,0.696000,0.698000,0.001549,0.002000
80,0.796,0.793,0.796,0.795,0.796,0.798,0.795000,0.796333,0.001633,0.001333
90,0.893,0.895,0.898,0.896,0.896,0.895,0.895333,0.895667,0.001643,0.000333
100,0.989,0.995,0.991,0.989,0.989,0.989,0.991667,0.989000,0.002422,0.002667


In [4]:
#Построение градуировочной характеристики
fig = go.Figure()

# Для хода поплавка вверх
fig.add_trace(go.Scatter(
    x=new_data['mean_up'],
    y=new_data.index,
    name='Прямой ход',
    mode='lines+markers'
))

# Для хода поплавка вниз
fig.add_trace(go.Scatter(
    x=new_data['mean_down'],
    y=new_data.index,
    name='Обратный ход',
    mode='lines'
))

# Идеальные показатели уровня жидкости в резервуаре(по показателям линейки)
fig.add_trace(go.Scatter(
    x=pd.Series(new_data.index).apply(lambda x: int(x)/100),
    y=new_data.index,
    name='Идеальная характеристика',
    mode='markers',
    marker=dict(size=8)
))

fig.update_layout(
    title='Градуировочная характеристика поплавкового уровнемера',
    yaxis_title='Уровень измеренный линейкой (см)',
    xaxis_title='Показания уровнемера (м)',
    legend_orientation='h',
    hovermode='x',
    width=800, height=1000,
)

fig.show()
fig.write_html('plotly/floatLevelGaugeChar.html')